In [1]:
import pandas as pd
import json
import os

with open('DICCIONARIO_CODIGO_NOMBRE_FARMACOS.json', 'r', encoding = 'utf-8') as f:
    DICCIONARIO_CODIGO_NOMBRE_FARMACOS = json.load(f)

In [3]:
class Formateador():
    def __init__(self):
        pass

    def obtener_entradas_todos_los_pacientes(self):
        entradas_todos_los_pacientes = []

        for nombre_archivo in os.listdir():
            if '.xls' in nombre_archivo:
                entradas_de_un_paciente = self.obtener_entradas_de_un_paciente(nombre_archivo)
                entradas_todos_los_pacientes += entradas_de_un_paciente

        return entradas_todos_los_pacientes

    def obtener_entradas_de_un_paciente(self, nombre_archivo):
        entradas = []

        datos_persona = self.obtener_datos_demograficos_de_un_paciente(nombre_archivo)
        lista_antibiogramas_persona = self.obtener_antibiogramas_de_un_paciente(nombre_archivo)
        lista_contaminados_persona = self.obtener_contaminados_de_un_paciente(nombre_archivo)
        lista_hongos_persona = self.obtener_cultivos_de_hongos_de_un_paciente(nombre_archivo)

        for antibiograma in lista_antibiogramas_persona:
            entrada_antibiograma = datos_persona + antibiograma
            entradas.append(entrada_antibiograma)
        
        for contaminados in lista_contaminados_persona:
            entrada_contaminados = datos_persona + contaminados
            entradas.append(entrada_contaminados)
        
        for hongos in lista_hongos_persona:
            entrada_hongos = datos_persona + hongos
            entradas.append(entrada_hongos)
        
        return entradas

    
    def obtener_antibiogramas_de_un_paciente(self, nombre_archivo):
        datos_totales = pd.read_excel(nombre_archivo)
        
        # indice_fila_inicio_antibio, indice_fila_termino_antibio = self.identificar_localizacion_antibiograma()
        antibiograma_crudo = datos_totales.iloc[18:31, :-2]
        antibiograma_formateado = self.formatear_tabla_antibiograma(antibiograma_crudo)
        lista_cepas_formato_df = self.separar_cepas(antibiograma_formateado)
        lista_cepas_formato_listas = list(map(self.mappear_resultados_a_formato_excel, lista_cepas_formato_df))

        return lista_cepas_formato_listas


    def formatear_tabla_antibiograma(self, antibiograma_crudo):
        antibiograma_formateado = antibiograma_crudo.copy()

        headers = ['ANTIBIOTICO']
        self.cantidad_de_cepas = int((len(antibiograma_formateado.columns) - 1)/ 2)
        for i in range(self.cantidad_de_cepas):
            headers.append(f'Cepa')
            headers.append(f'CIM')

        antibiograma_formateado.columns = headers
        antibiograma_formateado.ANTIBIOTICO = antibiograma_formateado.ANTIBIOTICO.map(DICCIONARIO_CODIGO_NOMBRE_FARMACOS)
        antibiograma_formateado.set_index(antibiograma_formateado.ANTIBIOTICO, inplace = True)
        antibiograma_formateado.drop(columns = 'ANTIBIOTICO', inplace = True)

        return antibiograma_formateado
    
    def separar_cepas(self, antibiograma_formateado):
        tablas_cepas = []
        for i in range(self.cantidad_de_cepas):
            df = antibiograma_formateado.iloc[:, i * 2: (i * 2) + 2]
            df = df[df['Cepa'].notna()]
            tablas_cepas.append(df)
        
        return tablas_cepas
    
    def mappear_resultados_a_formato_excel(tabla_una_cepa):
        diccionario_sensibilidades_a_llenar = {farmaco: None for farmaco in DICCIONARIO_CODIGO_NOMBRE_FARMACOS.values()}
        for farmaco in tabla_una_cepa.index:
            resultado_sensibilidad = tabla_una_cepa['Cepa'][farmaco]
            diccionario_sensibilidades_a_llenar[farmaco] = resultado_sensibilidad
        
        lista_sensibilidades_llenas = list(diccionario_sensibilidades_a_llenar.values())
        return lista_sensibilidades_llenas 


    def obtener_contaminados_de_un_paciente(self, nombre_archivo):
        contaminados = []

        return contaminados

    def obtener_cultivos_de_hongos_de_un_paciente(self, nombre_archivo):
        hongos = []

        return hongos

    def obtener_datos_demograficos_de_un_paciente(self, nombre_archivo):
        return datos_paciente


